In [1]:
import sys
root = '../../'
sys.path.append(root)   # Done to be able to import the packages and functions

import Utils.hico_evaluation.evaluation as ev
from Utils.custom_sampler import OverSampler
from Utils.custom_loss import MaskedBCELoss
from Utils.annotation_preprocessing import _load_csv_to_tensor
from Utils.train_val_split import train_val_split_hico
from hoi_classifiers import ElementwiseAvgFusionModel

import torch
import matplotlib.pyplot as plt

import random

import os
import shutil

In [2]:
seed = 42   #note that the model parameters will still be randomly initiated
torch.manual_seed(seed)
random.seed(seed)

#### Loading the data:

In [3]:
X_test = torch.load(root + "Embeddings/Combined_Embeddings/test.pt")
y_test = _load_csv_to_tensor(root + "anno/added/anno_augmented_test.csv").T[:,:600] # Transpose to make both first dimensions the #samples.
y_test[y_test.isnan()] = -1

In [4]:
test_paths = [root + "../hico_20150920/images/test2015/" + fname for fname in os.listdir(root + "../hico_20150920/images/test2015")]

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#### Loading the models:

In [6]:
visual_baseline_model = torch.load(root + "Saved Models/visual_baseline_model.pt").to(device)
bestmodel_67epochs_model = torch.load(root + "/Saved Models/best_model_67_epochs.pt").to(device)

#### use the models for inference on the test set:

In [7]:
output_visual_baseline = visual_baseline_model(X_test.to(device)[:,0,:])
output_bestmodel_67epochs = bestmodel_67epochs_model(X_test.to(device))

#### Checking the number images where all annotations are correctly predicted (ignoring the multitask learning classes):

In [8]:
visual_baseline_acc =(torch.where(((output_visual_baseline[:,:600] >= 0.5).int() == (y_test == 1).to(device)).all(dim=1))[0].shape[0] / y_test.to(device).shape[0]) * 100
best_model_67epochs_acc =(torch.where(((output_bestmodel_67epochs[:,:600] >= 0.5).int() == (y_test == 1).to(device)).all(dim=1))[0].shape[0] / y_test.to(device).shape[0]) * 100

print(f"Visual baseline Accuracy:                   {visual_baseline_acc}")
print(f"Elementwise Averaging (67 epochs) Accuracy: {best_model_67epochs_acc}")

Visual baseline Accuracy:                   25.440049699730793
Elementwise Averaging (67 epochs) Accuracy: 31.186581072685854


#### Looking at which samples are correctly classified by each model

In [9]:
correct_samples_visual_baseline =torch.where(((output_visual_baseline[:,:600] >= 0.5).int() == (y_test == 1).to(device)).all(dim=1))[0]
correct_samples_best_model_67epochs =torch.where(((output_bestmodel_67epochs[:,:600] >= 0.5).int() == (y_test == 1).to(device)).all(dim=1))[0]

In [10]:
corbaseline_in_corbest =torch.isin(correct_samples_visual_baseline, correct_samples_best_model_67epochs)

In [11]:
corbest_in_corbaseline =torch.isin(correct_samples_best_model_67epochs,correct_samples_visual_baseline)

#### Samples correctly by the baseline but not anymore with the elementwise average fusion:

In [12]:
sample_ids_misses_elementwiseavg =correct_samples_visual_baseline[torch.where(corbaseline_in_corbest == False)[0]]

In [13]:
#for path in [test_paths[i] for i in sample_ids_misses_elementwiseavg]:
    #shutil.copy(path, f"../../../missed_by_elementwiseAvg/{path.split('../../../hico_20150920/images/test2015/')[1]}")

Randomly selecting 4 images:

In [14]:
randomindices1 = torch.randint(len(sample_ids_misses_elementwiseavg),(4,))
[test_paths[i] for i in sample_ids_misses_elementwiseavg[randomindices1]]

['../../../hico_20150920/images/test2015/HICO_test2015_00002481.jpg',
 '../../../hico_20150920/images/test2015/HICO_test2015_00006636.jpg',
 '../../../hico_20150920/images/test2015/HICO_test2015_00002526.jpg',
 '../../../hico_20150920/images/test2015/HICO_test2015_00000053.jpg']

In [15]:
for i in sample_ids_misses_elementwiseavg[randomindices1]:
    print(torch.where((output_bestmodel_67epochs[i,:600] >= 0.5) != (y_test[i] == 1).to(device)))


(tensor([385], device='cuda:0'),)
(tensor([131, 141], device='cuda:0'),)
(tensor([94], device='cuda:0'),)
(tensor([467, 470, 472, 478, 479, 480, 481], device='cuda:0'),)


In [16]:
sample_ids_misses_elementwiseavg[randomindices1]

tensor([2427, 6532, 2470,   52], device='cuda:0')

In [17]:
(output_bestmodel_67epochs[52,:600] >= 0.5)[481]

tensor(True, device='cuda:0')

In [18]:
(y_test == 1)[52][467]

tensor(True)

#### Samples correctly by the elementwise average fusion but not by baseline:

In [19]:
sample_ids_misses_baseline =correct_samples_best_model_67epochs[torch.where(corbest_in_corbaseline == False)[0]]

In [20]:
#for path in [test_paths[i] for i in sample_ids_misses_baseline]:
    #shutil.copy(path, f"../../../missed_by_baseline/{path.split('../../../hico_20150920/images/test2015/')[1]}")

Randomly selecting 4 images:


In [21]:
randomindices2 = torch.randint(len(sample_ids_misses_baseline),(4,))
[test_paths[i] for i in sample_ids_misses_baseline[randomindices2]]

['../../../hico_20150920/images/test2015/HICO_test2015_00003349.jpg',
 '../../../hico_20150920/images/test2015/HICO_test2015_00005927.jpg',
 '../../../hico_20150920/images/test2015/HICO_test2015_00006241.jpg',
 '../../../hico_20150920/images/test2015/HICO_test2015_00008595.jpg']

In [22]:
for i in sample_ids_misses_baseline[randomindices2]:
    print(torch.where((output_visual_baseline[i,:600] >= 0.5) != (y_test[i] == 1).to(device)))

(tensor([589], device='cuda:0'),)
(tensor([370, 371, 374], device='cuda:0'),)
(tensor([470, 471, 472, 478], device='cuda:0'),)
(tensor([377, 380], device='cuda:0'),)


In [23]:
sample_ids_misses_baseline[randomindices2]

tensor([3275, 5823, 6137, 8489], device='cuda:0')

In [39]:
(output_visual_baseline[8489,:600] >= 0.5)[380]

tensor(True, device='cuda:0')

In [43]:
torch.where(y_test[8489] == 1)

(tensor([266]),)

In [38]:
test_paths[8489]

'../../../hico_20150920/images/test2015/HICO_test2015_00008595.jpg'